In [2]:
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow as tf
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import json
import nltk
from nltk.corpus import stopwords
from unidecode import unidecode
nltk.download("punkt")
from keybert import KeyBERT

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\port4\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
tokenizer = AutoTokenizer.from_pretrained("erst/xlm-roberta-base-finetuned-nace")
model = AutoModelForSequenceClassification.from_pretrained("erst/xlm-roberta-base-finetuned-nace")

pl = pipeline(
    "sentiment-analysis",
    model=model,
    tokenizer=tokenizer,
    return_all_scores=False,
)

pl("Country * Select a country... Afghanistan Åland Islands Albania Algeria American Samoa Andorra Angola Anguilla Antarctica Antigua and Barbuda Argentina Armenia Aruba Australia Austria Azerbaijan Bahamas Bahrain Bangladesh Barbados Belarus Belau Belgium Belize Benin Bermuda Bhutan Bolivia Bonaire, Saint Eustatius and Saba Bosnia and Herzegovina Botswana Bouvet Island Brazil British Indian Ocean Territory Brunei Bulgaria Burkina Faso Burundi Cambodia Cameroon Canada Cape Verde Cayman Islands Central African Republic Chad Chile China Christmas Island Cocos (Keeling) Islands Colombia Comoros Congo (Brazzaville) Congo (Kinshasa) Cook Islands Costa Rica Croatia Cuba Curaçao Cyprus Czech Republic Denmark Djibouti Dominica Dominican Republic Ecuador Egypt El Salvador Equatorial Guinea Eritrea Estonia Eswatini Ethiopia Falkland Islands Faroe Islands Fiji Finland France French Guiana French Polynesia French Southern Territories Gabon Gambia Georgia Germany Ghana Gibraltar Greece Greenland Grenada Guadeloupe Guam Guatemala Guernsey Guinea Guinea-Bissau Guyana Haiti Heard Island and McDonald Islands Honduras Hong Kong Hungary Iceland India Indonesia Iran Iraq Ireland Isle of Man Israel Italy Ivory Coast Jamaica Japan Jersey Jordan Kazakhstan Kenya Kiribati Kuwait Kyrgyzstan Laos Latvia Lebanon Lesotho Liberia Libya Liechtenstein Lithuania Luxembourg Macao Madagascar Malawi Malaysia Maldives Mali Malta Marshall Islands Martinique Mauritania Mauritius Mayotte Mexico Micronesia Moldova Monaco Mongolia Montenegro Montserrat Morocco Mozambique Myanmar Namibia Nauru Nepal Netherlands New Caledonia New Zealand Nicaragua Niger Nigeria Niue Norfolk Island North Korea North Macedonia Northern Mariana Islands Norway Oman Pakistan Palestinian Territory Panama Papua New Guinea Paraguay Peru Philippines Pitcairn Poland Portugal Puerto Rico Qatar Reunion Romania Russia Rwanda São Tomé and Príncipe Saint Barthélemy Saint Helena Saint Kitts and Nevis Saint Lucia Saint Martin (Dutch part) Saint Martin (French part) Saint Pierre and Miquelon Saint Vincent and the Grenadines Samoa San Marino Saudi Arabia Senegal Serbia Seychelles Sierra Leone Singapore Slovakia Slovenia Solomon Islands Somalia South Africa South Georgia/Sandwich Islands South Korea South Sudan Spain Sri Lanka Sudan Suriname Svalbard and Jan Mayen Sweden Switzerland Syria Taiwan Tajikistan Tanzania Thailand Timor-Leste Togo Tokelau Tonga Trinidad and")

[{'label': '8411', 'score': 0.5752618312835693}]

In [3]:
# Define HuggingFace model setup & keyBert
tokenizer = AutoTokenizer.from_pretrained("erst/xlm-roberta-base-finetuned-nace")
model = AutoModelForSequenceClassification.from_pretrained("erst/xlm-roberta-base-finetuned-nace")
pl = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, return_all_scores=False)
kw_model = KeyBERT()
kw_model_multi = KeyBERT('paraphrase-multilingual-MiniLM-L12-v2')
print("Models setup")

C:\Users\port4\IdeaProjects\AI-Webcrawler11\myenv\lib\site-packages\transformers\pipelines\text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


Models setup


In [ ]:
#####Ready for the scraped data in results.json##### Adjust Paths if older version of main branch
# Define file paths
json_file_path = "data/results.json"
json_print_path = "data/NACE_Interpreted.json"
bp_json_file_path = "data/bp(with codes 42K).json"
duns_json_file_path = "data/DUNS_BP.json"
test_json_file_path = "data/bp2(with 25 sites).json"
nace_codes_file_path = "NACE-Codes.json"

# Define JSON paths
path_mappings = {
    "Summary": ("Summary",),
    "Paragraphs": ("Paragraphs",),
    #"Sentences": ("AboutUs", "Sentence"),
    "Company_Name": ("name",),
    "Company_Key": ("key",),
    "WholeText": ("WholeText",),
}

# Load NACE codes
with open(nace_codes_file_path, 'r') as config_file:
    nace_codes = json.load(config_file)

# Function to get the description for a specific NACE code
def get_nace_description(nace_code):
    return nace_codes.get(nace_code, "NACE code not found")

# Function to clean German text TODO also english, spanish...
def clean_text(text):
    words = nltk.word_tokenize(text)
    stop_words = set(stopwords.words("german"))
    filtered_words = [word for word in words if word.lower() not in stop_words]
    cleaned_text = " ".join(filtered_words)
    cleaned_text = cleaned_text.replace("\n", " ").replace("\t", " ").replace("\\", "")
    return cleaned_text

# Function to process data for a specific type
def process_text(data, type_name):
    path = path_mappings[type_name]
    
    for item in data:
        content = item.get(path[0], "")[:700]
        comp_name = item.get(path_mappings["Company_Name"][0], "")
        comp_key = item.get(path_mappings["Company_Key"][0], "")
        if not content:
            continue
        cleaned_text = clean_text(content)
        #cleaned_text = cleanForLLM(kw_model.extract_keywords(content, top_n=10))
        print(cleaned_text)
        nace_result = pl(cleaned_text)[0]
        nace_score = nace_result["label"]
        
        page_with_id = {
            "type": type_name,
            "key": comp_key,
            "nace score HF": nace_score,
            "descriptor HF": get_nace_description(nace_score),
            "confidence HF": nace_result["score"],
            "name": comp_name
        }
        about_us_pages_with_ids.append(page_with_id)

def process_jsonarray(jsonarray, type, comp_name, comp_key):
    temp_pages = []
    
    for ele in jsonarray: #.values(): Delete # if you have to select the values
        try:
            words = ele.split()
            if (type == "Paragraphs" and 2 <= len(words) <= 320) or (type == "Sentences" and 1 < len(words)):
                #cleaned_text = cleanForLLM(kw_model.extract_keywords(ele, top_n=4))
                cleaned_text = clean_text(ele)[:700]
                nace_result = pl(cleaned_text)[0]
                nace_score = nace_result["label"]
                page_with_id = {
                    "nace score HF": nace_score,
                    "key": comp_key,
                    "descriptor HF": get_nace_description(nace_score),
                    "confidence HF": nace_result["score"],
                    "name": comp_name
                }
                temp_pages.append(page_with_id)
        except ValueError:
            print("Error calculating NACE")
    
    calculate_nace_stats(temp_pages, type, comp_key, comp_name)

def calculate_nace_stats(pages, type_name, comp_key, comp_name):
    if not pages:
        return

    nace_counts = {}
    confidence_sum = 0
    num_pages = len(pages)

    for page in pages:
        nace_score = page["nace score HF"]
        confidence = page["confidence HF"]

        if nace_score in nace_counts:
            nace_counts[nace_score] += 1
        else:
            nace_counts[nace_score] = 1
        
        confidence_sum += confidence

    try:
        most_common_nace = max(nace_counts, key=nace_counts.get)
        average_confidence = confidence_sum / num_pages
    except ValueError:
        print(f"Error calculating most common NACE Score for {type_name}")
        return
    
    page_with_id = {
        "type": type_name,
        "nace score HF": most_common_nace,
        "descriptor HF": get_nace_description(most_common_nace),
        "confidence HF": average_confidence,
        "key": comp_key,
        "name": comp_name
    }
    about_us_pages_with_ids.append(page_with_id)

def cleanForLLM(data):
    my_list = []  # Create an empty list to store the first characters
    for item in data:
        if item:  # Check if the string is not empty
            my_list.append(item[0])
    return ', '.join(my_list)

#########################################Evaluation#########################################

def rwNaceKey(nacecodeFieldName, keyFieldName, filepath, about_us_pages_with_ids):
    # Process BP data and update NACE Code System
    with open(filepath, 'r', encoding='utf8') as bp_json_file:
        bp_data = json.load(bp_json_file)
        for entry in bp_data:
            key = entry.get(keyFieldName)
            nace_code_system = entry.get(nacecodeFieldName)
            if key and nace_code_system:
                key_nace_mapping[key] = nace_code_system[0][-5:].replace(".", "")
    
    # Update NACE Code System for pages
    for page_with_id in about_us_pages_with_ids:
        key = page_with_id.get("key")
        if key in key_nace_mapping:
            page_with_id[nacecodeFieldName] = key_nace_mapping[key]
    return about_us_pages_with_ids

def calculate_difference(about_us_pages_with_ids, key):
    similarities_counts = {"Summary": 0, "Paragraphs": 0, "Sentences": 0, "WholeText": 0}
    dissimilarities_counts = {"Summary": 0, "Paragraphs": 0, "Sentences": 0, "WholeText": 0}
    for entry in about_us_pages_with_ids:

        nace_code = entry["nace score HF"]
        nace_code_system = entry.get(key)
        entry_type = entry["type"]
    
        if nace_code == nace_code_system:
            similarities_counts[entry_type] += 1
        else:
            dissimilarities_counts[entry_type] += 1
    return similarities_counts, dissimilarities_counts

#########################################Main#########################################

# Initialize data structures
key_nace_mapping = {}
about_us_pages_with_ids = []

# Process "summary" data
with open(json_file_path, 'r', encoding='utf8') as json_file:
    data = json.load(json_file)
    #process_text(data,"WholeText")
    process_text(data, "Summary")

# Process "paragraphs" and "sentence" data
for item in data:
    paragraphs = item.get(path_mappings["Paragraphs"][0], {})
    #sentences = item.get(path_mappings["Sentences"][0], {}).get(path_mappings["Sentences"][1], {})
    comp_name = item.get(path_mappings["Company_Name"][0], {})
    comp_key = item.get(path_mappings["Company_Key"][0], {})

    if not paragraphs and not sentences:
        continue
    print(item)
    if paragraphs:
        process_jsonarray(paragraphs, "Paragraphs", comp_name, comp_key)
    
    #if sentences:
     #   process_jsonarray(sentences, "Sentences", comp_name, comp_key)
    
about_us_pages_with_ids = rwNaceKey("industryCodeList", "key", bp_json_file_path, about_us_pages_with_ids)
about_us_pages_with_ids = rwNaceKey("dunsIndustryCodeList", "key", duns_json_file_path, about_us_pages_with_ids)

#print basic evaluation measure
print(calculate_difference(about_us_pages_with_ids, "dunsIndustryCodeList"))
    
# Save results to a JSON file
with open(json_print_path, 'w', encoding='utf8') as json_file:
    json.dump(about_us_pages_with_ids, json_file)

Zusammenspiel Top-Herstellern Branche überzeugen umfassenden Angebot Lagerware , kennen Trends , Möglichkeiten Bedeutung , Sortiment Heizung Umweltschutz bedeutet
com kostenfrei 11:00 11:30 Melina Breuer Stephan Rotthaus / rotthaus
Spezialisierung Bereich Beatmung Spezialisierung Bereich Beatmung Spezialisierung Bereich Beatmung breites Produktportfolio zahlreichen Komplettlösungen breites Produktportfolio zahlreichen Komplettlösungen breites Produktportfolio zahlreichen Komplettlösungen Fähigkeit , innovativ , effizient effektiv Kundenbedürfnisse reagieren
Visit your local site : Who we are Our brands Investor relations Governance News & Media Sustainability Who we are Investor relations Governance News & Media Sustainability
Energy needs to be distributed and sensors and actuators need to be connected across all these areas , and the best place to do that is one central location – the control panel
our Commitment Camurus is committed to lead the development of advanced drug delivery 

In [48]:
#####Ready for the manual data in data/bp2(with 25 sites).json#####
# Define file paths
json_file_path = "data/results.json"
json_print_path = "data/NACE_Interpreted.json"
bp_json_file_path = "data/bp(with codes 42K).json"
duns_json_file_path = "data/DUNS_BP.json"
test_json_file_path = "data/bp2(with 25 sites).json"
nace_codes_file_path = "NACE-Codes.json"

# Define JSON paths
path_mappings = {
    "Summary": ("AboutUs", "Summary",),
    "Paragraphs": ("AboutUs","Paragraph",),
    "Sentences": ("AboutUs", "Sentence"),
    "Company_Name": ("Company", "name",),
    "Company_Key": ("Company", "key",),
    "WholeText": ("AboutUs", "WholeText",),
}

# Load NACE codes
with open(nace_codes_file_path, 'r') as config_file:
    nace_codes = json.load(config_file)

# Function to get the description for a specific NACE code
def get_nace_description(nace_code):
    return nace_codes.get(nace_code, "NACE code not found")

# Function to clean German text TODO also english, spanish...
def clean_text(text):
    words = nltk.word_tokenize(text)
    stop_words = set(stopwords.words("german"))
    filtered_words = [word for word in words if word.lower() not in stop_words]
    cleaned_text = " ".join(filtered_words)
    cleaned_text = cleaned_text.replace("\n", " ").replace("\t", " ").replace("\\", "")
    return cleaned_text

# Function to process data for a specific type
def process_text(data, type_name):
    path = path_mappings[type_name]
    
    for item in data:
        content = item.get(path[0], "").get(path[1], "")
        comp_name = item.get(path_mappings["Company_Name"][0], "").get(path_mappings["Company_Name"][1], "")
        comp_key = item.get(path_mappings["Company_Key"][0], "").get(path_mappings["Company_Key"][1], "")
        if not content:
            continue
        cleaned_text = cleanForLLM(kw_model.extract_keywords(content, top_n=15))
        nace_result = pl(cleaned_text)[0]
        nace_score = nace_result["label"]
        
        page_with_id = {
            "type": type_name,
            "key": comp_key,
            "NACE Score": nace_score,
            "Descriptor": get_nace_description(nace_score),
            "Confidence": nace_result["score"],
            "Name": comp_name
        }
        about_us_pages_with_ids.append(page_with_id)

def process_jsonarray(jsonarray, type, comp_name, comp_key):
    temp_pages = []
    
    for ele in jsonarray.values(): #Delete # if you have to select the values
        try:
            words = ele.split()
            if (type == "Paragraphs" and 2 <= len(words) <= 320) or (type == "Sentences" and 1 < len(words)):
                cleaned_text = cleanForLLM(kw_model.extract_keywords(ele, top_n=3))
                nace_result = pl(cleaned_text)[0]
                nace_score = nace_result["label"]
                page_with_id = {
                    "NACE Score": nace_score,
                    "key": comp_key,
                    "Descriptor": get_nace_description(nace_score),
                    "Confidence": nace_result["score"],
                    "Name": comp_name
                }
                temp_pages.append(page_with_id)
        except ValueError:
            print("Error calculating NACE")
    
    calculate_nace_stats(temp_pages, type, comp_key, comp_name)

def calculate_nace_stats(pages, type_name, comp_key, comp_name):
    if not pages:
        return

    nace_counts = {}
    confidence_sum = 0
    num_pages = len(pages)

    for page in pages:
        nace_score = page["NACE Score"]
        confidence = page["Confidence"]

        if nace_score in nace_counts:
            nace_counts[nace_score] += 1
        else:
            nace_counts[nace_score] = 1
        
        confidence_sum += confidence

    try:
        most_common_nace = max(nace_counts, key=nace_counts.get)
        average_confidence = confidence_sum / num_pages
    except ValueError:
        print(f"Error calculating most common NACE Score for {type_name}")
        return
    
    page_with_id = {
        "type": type_name,
        "NACE Score": most_common_nace,
        "Descriptor": get_nace_description(most_common_nace),
        "Confidence": average_confidence,
        "key": comp_key,
        "Name": comp_name
    }
    about_us_pages_with_ids.append(page_with_id)

def cleanForLLM(data):
    my_list = []  # Create an empty list to store the first characters
    for item in data:
        if item:  # Check if the string is not empty
            my_list.append(item[0])
    return ', '.join(my_list)

#########################################Evaluation#########################################

def rwNaceKey(nacecodeFieldName, keyFieldName, filepath, about_us_pages_with_ids):
    # Process BP data and update NACE Code System
    with open(filepath, 'r', encoding='utf8') as bp_json_file:
        bp_data = json.load(bp_json_file)
        for entry in bp_data:
            key = entry.get(keyFieldName)
            nace_code_system = entry.get(nacecodeFieldName)
            if key and nace_code_system:
                key_nace_mapping[key] = nace_code_system[0][-5:].replace(".", "")
    
    # Update NACE Code System for pages
    for page_with_id in about_us_pages_with_ids:
        key = page_with_id.get("key")
        if key in key_nace_mapping:
            page_with_id[nacecodeFieldName] = key_nace_mapping[key]
    return about_us_pages_with_ids

def calculate_difference(about_us_pages_with_ids, key):
    similarities_counts = {"Summary": 0, "Paragraphs": 0, "Sentences": 0, "WholeText": 0}
    dissimilarities_counts = {"Summary": 0, "Paragraphs": 0, "Sentences": 0, "WholeText": 0}
    for entry in about_us_pages_with_ids:

        nace_code = entry["NACE Score"]
        nace_code_system = entry.get(key)
        entry_type = entry["type"]
    
        if nace_code == nace_code_system:
            similarities_counts[entry_type] += 1
        else:
            dissimilarities_counts[entry_type] += 1
    return similarities_counts, dissimilarities_counts

#########################################Main#########################################

# Initialize data structures
key_nace_mapping = {}
about_us_pages_with_ids = []

# Process "summary" data
with open(test_json_file_path, 'r', encoding='utf8') as json_file:
    data = json.load(json_file)
    process_text(data, "WholeText")
    process_text(data, "Summary")

# Process "paragraphs" and "sentence" data
for item in data:
    paragraphs = item.get(path_mappings["Paragraphs"][0], {}).get(path_mappings["Paragraphs"][1], {})
    sentences = item.get(path_mappings["Sentences"][0], {}).get(path_mappings["Sentences"][1], {})
    comp_name = item.get(path_mappings["Company_Name"][0], "").get(path_mappings["Company_Name"][1], "")
    comp_key = item.get(path_mappings["Company_Key"][0], "").get(path_mappings["Company_Key"][1], "")

    if not paragraphs and not sentences:
        continue
    
    if paragraphs:
        process_jsonarray(paragraphs, "Paragraphs", comp_name, comp_key)
    
    if sentences:
        process_jsonarray(sentences, "Sentences", comp_name, comp_key)
    
about_us_pages_with_ids = rwNaceKey("industryCodeList", "key", bp_json_file_path, about_us_pages_with_ids)
about_us_pages_with_ids = rwNaceKey("dunsIndustryCodeList", "key", duns_json_file_path, about_us_pages_with_ids)

#print basic evaluation measure
print(calculate_difference(about_us_pages_with_ids, "industryCodeList"))
print(calculate_difference(about_us_pages_with_ids, "dunsIndustryCodeList"))
    
# Save results to a JSON file
with open(json_print_path, 'w', encoding='utf8') as json_file:
    json.dump(about_us_pages_with_ids, json_file)

({'Summary': 2, 'Paragraphs': 0, 'Sentences': 2, 'WholeText': 2}, {'Summary': 23, 'Paragraphs': 25, 'Sentences': 23, 'WholeText': 23})
({'Summary': 2, 'Paragraphs': 0, 'Sentences': 2, 'WholeText': 2}, {'Summary': 23, 'Paragraphs': 25, 'Sentences': 23, 'WholeText': 23})
